In [1]:
import findspark
findspark.init()
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

# DataFrame 변환

In [2]:
p = [[1, [1.0, 2.0, 3.0]], [1, [1.1, 2.1, 3.1]], [0, [1.2, 2.2, 3.3]]]

In [3]:
print ("label: {}\nfeatures: {}".format(p[0][0], p[0][1]))

label: 1
features: [1.0, 2.0, 3.0]


In [4]:
trainDf=spark.createDataFrame(p)

In [5]:
trainDf.collect()

[Row(_1=1, _2=[1.0, 2.0, 3.0]),
 Row(_1=1, _2=[1.1, 2.1, 3.1]),
 Row(_1=0, _2=[1.2, 2.2, 3.3])]

In [6]:
from pyspark.mllib.regression import LabeledPoint
p = [LabeledPoint(1, [1.0,2.0,3.0]),
     LabeledPoint(1, [1.1,2.1,3.1]),
     LabeledPoint(0, [1.2,2.2,3.3])]

In [7]:
trainDf=spark.createDataFrame(p)

In [8]:
trainDf.collect()

[Row(features=DenseVector([1.0, 2.0, 3.0]), label=1.0),
 Row(features=DenseVector([1.1, 2.1, 3.1]), label=1.0),
 Row(features=DenseVector([1.2, 2.2, 3.3]), label=0.0)]

In [9]:
from pyspark.mllib.linalg import Vectors

trainDf = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, 1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, 0.5]))], ["label", "features"])

In [10]:
trainDf.collect()

[Row(label=1.0, features=DenseVector([0.0, 1.1, 0.1])),
 Row(label=0.0, features=DenseVector([2.0, 1.0, 1.0])),
 Row(label=0.0, features=DenseVector([2.0, 1.3, 1.0])),
 Row(label=1.0, features=DenseVector([0.0, 1.2, 0.5]))]

In [11]:
from pyspark.ml.linalg import SparseVector # ml ok

_rdd = spark.sparkContext.parallelize([
    (0.0, SparseVector(4, {1: 1.0, 3: 5.5})),
    (1.0, SparseVector(4, {0: -1.0, 2: 0.5}))])

In [12]:
_df=_rdd.toDF()

In [13]:
_df=_df.withColumnRenamed('_1', 'label').withColumnRenamed('_2', 'features')

In [14]:
_df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0| (4,[1,3],[1.0,5.5])|
|  1.0|(4,[0,2],[-1.0,0.5])|
+-----+--------------------+



# 단어 빈도

In [15]:
doc=[
    "When I find myself in times of trouble",
    "Mother Mary comes to me",
    "Speaking words of wisdom, let it be",
    "And in my hour of darkness",
    "She is standing right in front of me",
    "Speaking words of wisdom, let it be",
    "Let it be",
    "Let it be",
    "Let it be",
    "Let it be",
    "Whisper words of wisdom, let it be"
]

In [16]:
d={}
for sentence in doc:
    words=sentence.split()
    for word in words:
        if word in d:
            d[word]+=1
        else:
            d[word]=1

In [17]:
for k,v in d.items():
    print ("{}\t{}".format(k,v))

When	1
I	1
find	1
myself	1
in	3
times	1
of	6
trouble	1
Mother	1
Mary	1
comes	1
to	1
me	2
Speaking	2
words	3
wisdom,	3
let	3
it	7
be	7
And	1
my	1
hour	1
darkness	1
She	1
is	1
standing	1
right	1
front	1
Let	4
Whisper	1


In [18]:
doc2d=[
    ["When I find myself in times of trouble"],
    ["Mother Mary comes to me"],
    ["Speaking words of wisdom, let it be"],
    ["And in my hour of darkness"],
    ["She is standing right in front of me"],
    ["Speaking words of wisdom, let it be"],
    [u"우리 Let it be"],
    [u"나 Let it be"],
    [u"너 Let it be"],
    ["Let it be"],
    ["Whisper words of wisdom, let it be"]
]


In [19]:
myDf=spark.createDataFrame(doc2d, ['sent'])

In [20]:
myDf.show(truncate=True)

+--------------------+
|                sent|
+--------------------+
|When I find mysel...|
|Mother Mary comes...|
|Speaking words of...|
|And in my hour of...|
|She is standing r...|
|Speaking words of...|
|      우리 Let it be|
|        나 Let it be|
|        너 Let it be|
|           Let it be|
|Whisper words of ...|
+--------------------+



In [21]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="words")

In [22]:
tokDf = tokenizer.transform(myDf)

In [23]:
tokDf.show(3)

+--------------------+--------------------+
|                sent|               words|
+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|
|Mother Mary comes...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|
+--------------------+--------------------+
only showing top 3 rows



In [24]:
from pyspark.ml.feature import RegexTokenizer

re = RegexTokenizer(inputCol="sent", outputCol="wordsReg", pattern="\\s+")

In [25]:
reDf=re.transform(myDf)
reDf.show()

+--------------------+--------------------+
|                sent|            wordsReg|
+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|
|Mother Mary comes...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|
|She is standing r...|[she, is, standin...|
|Speaking words of...|[speaking, words,...|
|      우리 Let it be| [우리, let, it, be]|
|        나 Let it be|   [나, let, it, be]|
|        너 Let it be|   [너, let, it, be]|
|           Let it be|       [let, it, be]|
|Whisper words of ...|[whisper, words, ...|
+--------------------+--------------------+



In [26]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="wordsReg", outputCol="nostops")

In [27]:
stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

In [28]:
_mystopwords=[u"나",u"너", u"우리"]
for e in _mystopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)

StopWordsRemover_118ec901a7d8

In [29]:
for e in stop.getStopWords():
    print (e, end="/")

i/me/my/myself/we/our/ours/ourselves/you/your/yours/yourself/yourselves/he/him/his/himself/she/her/hers/herself/it/its/itself/they/them/their/theirs/themselves/what/which/who/whom/this/that/these/those/am/is/are/was/were/be/been/being/have/has/had/having/do/does/did/doing/a/an/the/and/but/if/or/because/as/until/while/of/at/by/for/with/about/against/between/into/through/during/before/after/above/below/to/from/up/down/in/out/on/off/over/under/again/further/then/once/here/there/when/where/why/how/all/any/both/each/few/more/most/other/some/such/no/nor/not/only/own/same/so/than/too/very/s/t/can/will/just/don/should/now/i'll/you'll/he'll/she'll/we'll/they'll/i'd/you'd/he'd/she'd/we'd/they'd/i'm/you're/he's/she's/it's/we're/they're/i've/we've/you've/they've/isn't/aren't/wasn't/weren't/haven't/hasn't/hadn't/don't/doesn't/didn't/won't/wouldn't/shan't/shouldn't/mustn't/can't/couldn't/cannot/could/here's/how's/let's/ought/that's/there's/what's/when's/where's/who's/why's/would/나/너/우리/

In [30]:
stopDf=stop.transform(reDf)
stopDf.show()

+--------------------+--------------------+--------------------+
|                sent|            wordsReg|             nostops|
+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[find, times, tro...|
|Mother Mary comes...|[mother, mary, co...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|    [hour, darkness]|
|She is standing r...|[she, is, standin...|[standing, right,...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|
|      우리 Let it be| [우리, let, it, be]|               [let]|
|        나 Let it be|   [나, let, it, be]|               [let]|
|        너 Let it be|   [너, let, it, be]|               [let]|
|           Let it be|       [let, it, be]|               [let]|
|Whisper words of ...|[whisper, words, ...|[whisper, words, ...|
+--------------------+--------------------+--------------------+



In [31]:
from functools import reduce
doc = reduce(lambda x,y: x+y, doc2d)

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')

In [33]:
print (vectorizer.fit_transform(doc))

  (0, 9)	1
  (0, 10)	1
  (1, 5)	1
  (1, 4)	1
  (1, 0)	1
  (2, 7)	1
  (2, 13)	1
  (2, 12)	1
  (2, 3)	1
  (3, 2)	1
  (3, 1)	1
  (4, 8)	1
  (4, 6)	1
  (5, 7)	1
  (5, 13)	1
  (5, 12)	1
  (5, 3)	1
  (6, 3)	1
  (6, 14)	1
  (7, 3)	1
  (8, 3)	1
  (9, 3)	1
  (10, 13)	1
  (10, 12)	1
  (10, 3)	1
  (10, 11)	1


In [34]:
vectorizer.vocabulary_

{'times': 9,
 'trouble': 10,
 'mother': 5,
 'mary': 4,
 'comes': 0,
 'speaking': 7,
 'words': 13,
 'wisdom': 12,
 'let': 3,
 'hour': 2,
 'darkness': 1,
 'standing': 8,
 'right': 6,
 '우리': 14,
 'whisper': 11}

In [35]:
vectorizer.fit_transform(doc).todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0],
        [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]], dtype=int64)

In [36]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="nostops", outputCol="cv", vocabSize=30, minDF=1.0)

In [37]:
cvModel = cv.fit(stopDf)

In [38]:
print (type(cv),type(cvModel))

<class 'pyspark.ml.feature.CountVectorizer'> <class 'pyspark.ml.feature.CountVectorizerModel'>


In [39]:
cvDf = cvModel.transform(stopDf)

In [40]:
cvDf.drop('wordsReg').show(3)

+--------------------+--------------------+--------------------+
|                sent|             nostops|                  cv|
+--------------------+--------------------+--------------------+
|When I find mysel...|[find, times, tro...|(16,[5,6,8],[1.0,...|
|Mother Mary comes...|[mother, mary, co...|(16,[10,13,14],[1...|
|Speaking words of...|[speaking, words,...|(16,[0,1,2,3],[1....|
+--------------------+--------------------+--------------------+
only showing top 3 rows



In [41]:
import math

tf=1./4
df=3.
N=11.
idf=math.log((N+1)/(df+1))+1
print ("idf: {}".format(idf))

idf: 2.09861228866811


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=1.0, stop_words='english',norm = None)

In [43]:
print (vectorizer.fit_transform(doc))

  (0, 10)	2.791759469228055
  (0, 9)	2.791759469228055
  (1, 0)	2.791759469228055
  (1, 4)	2.791759469228055
  (1, 5)	2.791759469228055
  (2, 3)	1.4054651081081644
  (2, 12)	2.09861228866811
  (2, 13)	2.09861228866811
  (2, 7)	2.386294361119891
  (3, 1)	2.791759469228055
  (3, 2)	2.791759469228055
  (4, 6)	2.791759469228055
  (4, 8)	2.791759469228055
  (5, 3)	1.4054651081081644
  (5, 12)	2.09861228866811
  (5, 13)	2.09861228866811
  (5, 7)	2.386294361119891
  (6, 14)	2.791759469228055
  (6, 3)	1.4054651081081644
  (7, 3)	1.4054651081081644
  (8, 3)	1.4054651081081644
  (9, 3)	1.4054651081081644
  (10, 11)	2.791759469228055
  (10, 3)	1.4054651081081644
  (10, 12)	2.09861228866811
  (10, 13)	2.09861228866811


In [44]:
vectorizer.vocabulary_

{'times': 9,
 'trouble': 10,
 'mother': 5,
 'mary': 4,
 'comes': 0,
 'speaking': 7,
 'words': 13,
 'wisdom': 12,
 'let': 3,
 'hour': 2,
 'darkness': 1,
 'standing': 8,
 'right': 6,
 '우리': 14,
 'whisper': 11}

In [45]:
vectorizer.idf_

array([2.79175947, 2.79175947, 2.79175947, 1.40546511, 2.79175947,
       2.79175947, 2.79175947, 2.38629436, 2.79175947, 2.79175947,
       2.79175947, 2.79175947, 2.09861229, 2.09861229, 2.79175947])

In [46]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash")

In [47]:
hashDf = hashTF.transform(stopDf)

In [48]:
hashDf.select("hash").show(truncate=False)

+--------------------------------------------------------+
|hash                                                    |
+--------------------------------------------------------+
|(262144,[64317,91878,152481],[1.0,1.0,1.0])             |
|(262144,[24657,63767,245426],[1.0,1.0,1.0])             |
|(262144,[27556,151864,173339,175131],[1.0,1.0,1.0,1.0]) |
|(262144,[74517,98431],[1.0,1.0])                        |
|(262144,[84798,218360,229166],[1.0,1.0,1.0])            |
|(262144,[27556,151864,173339,175131],[1.0,1.0,1.0,1.0]) |
|(262144,[173339],[1.0])                                 |
|(262144,[173339],[1.0])                                 |
|(262144,[173339],[1.0])                                 |
|(262144,[173339],[1.0])                                 |
|(262144,[151864,173339,175131,188139],[1.0,1.0,1.0,1.0])|
+--------------------------------------------------------+



In [49]:
a = hashDf.select("hash").rdd.map(lambda x:x[0]).collect()

In [51]:
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="words", outputCol="w2v")
model = word2Vec.fit(tokDf)
w2vDf = model.transform(tokDf)

In [52]:
model.getVectors().show(truncate=False)

+--------+------------------------------------------------------------------+
|word    |vector                                                            |
+--------+------------------------------------------------------------------+
|trouble |[0.08801643550395966,0.03672950714826584,-0.1287679374217987]     |
|mother  |[0.11748811602592468,-0.02989991381764412,-0.11452135443687439]   |
|find    |[-0.13126887381076813,-0.027504412457346916,0.07296513020992279]  |
|standing|[-0.06447822600603104,-0.020746180787682533,0.11937719583511353]  |
|wisdom, |[-0.1087152287364006,0.045862145721912384,-0.1298976093530655]    |
|in      |[0.11704662442207336,0.07607141882181168,-0.08325448632240295]    |
|myself  |[-0.05106130614876747,-0.07791944593191147,-0.15875688195228577]  |
|is      |[-0.04754877835512161,-0.02113082818686962,-0.0039343019016087055]|
|darkness|[-0.1577366590499878,0.0678299218416214,-0.09224570542573929]     |
|우리    |[-0.030192559584975243,-0.07797732949256897,-0.118030525

In [55]:
model.findSynonyms("she", 3).show()

+--------+------------------+
|    word|        similarity|
+--------+------------------+
|      it|0.9208083748817444|
|darkness|0.8202618956565857|
| wisdom,|0.8098458051681519|
+--------+------------------+



In [56]:
from pyspark.ml.feature import NGram

ngram = NGram(n=2, inputCol="words", outputCol="ngrams")

In [57]:
ngramDf = ngram.transform(tokDf)

In [58]:
ngramDf.show()

+--------------------+--------------------+----------------------+
|                sent|               words|                ngrams|
+--------------------+--------------------+----------------------+
|When I find mysel...|[when, i, find, m...|  [when i, i find, ...|
|Mother Mary comes...|[mother, mary, co...|  [mother mary, mar...|
|Speaking words of...|[speaking, words,...|  [speaking words, ...|
|And in my hour of...|[and, in, my, hou...|  [and in, in my, m...|
|She is standing r...|[she, is, standin...|  [she is, is stand...|
|Speaking words of...|[speaking, words,...|  [speaking words, ...|
|      우리 Let it be| [우리, let, it, be]|[우리 let, let it, ...|
|        나 Let it be|   [나, let, it, be]| [나 let, let it, i...|
|        너 Let it be|   [너, let, it, be]| [너 let, let it, i...|
|           Let it be|       [let, it, be]|       [let it, it be]|
|Whisper words of ...|[whisper, words, ...|  [whisper words, w...|
+--------------------+--------------------+----------------------+



In [59]:
for e in ngramDf.select("words","ngrams").take(3):
    print (e)

Row(words=['when', 'i', 'find', 'myself', 'in', 'times', 'of', 'trouble'], ngrams=['when i', 'i find', 'find myself', 'myself in', 'in times', 'times of', 'of trouble'])
Row(words=['mother', 'mary', 'comes', 'to', 'me'], ngrams=['mother mary', 'mary comes', 'comes to', 'to me'])
Row(words=['speaking', 'words', 'of', 'wisdom,', 'let', 'it', 'be'], ngrams=['speaking words', 'words of', 'of wisdom,', 'wisdom, let', 'let it', 'it be'])


In [60]:
from pyspark.ml.feature import StringIndexer

labelIndexer = StringIndexer(inputCol="sent", outputCol="sentLabel")

In [61]:
model=labelIndexer.fit(myDf)

In [62]:
siDf=model.transform(myDf)

In [63]:
siDf.show()

+--------------------+---------+
|                sent|sentLabel|
+--------------------+---------+
|When I find mysel...|      5.0|
|Mother Mary comes...|      3.0|
|Speaking words of...|      0.0|
|And in my hour of...|      1.0|
|She is standing r...|      4.0|
|Speaking words of...|      0.0|
|      우리 Let it be|      9.0|
|        나 Let it be|      7.0|
|        너 Let it be|      8.0|
|           Let it be|      2.0|
|Whisper words of ...|      6.0|
+--------------------+---------+



In [65]:
%%writefile data/ds_spark_heightweight.txt
1   65.78   112.99
2   71.52   136.49
3   69.40   153.03
4   68.22   142.34
5   67.79   144.30
6   68.70   123.30
7   69.80   141.49
8   70.01   136.46
9   67.90   112.37
10  66.78   120.67
11  66.49   127.45
12  67.62   114.14
13  68.30   125.61
14  67.12   122.46
15  68.28   116.09
16  71.09   140.00
17  66.46   129.50
18  68.65   142.97
19  71.23   137.90
20  67.13   124.04
21  67.83   141.28
22  68.88   143.54
23  63.48   97.90
24  68.42   129.50
25  67.63   141.85
26  67.21   129.72
27  70.84   142.42
28  67.49   131.55
29  66.53   108.33
30  65.44   113.89
31  69.52   103.30
32  65.81   120.75
33  67.82   125.79
34  70.60   136.22
35  71.80   140.10
36  69.21   128.75
37  66.80   141.80
38  67.66   121.23
39  67.81   131.35
40  64.05   106.71
41  68.57   124.36
42  65.18   124.86
43  69.66   139.67
44  67.97   137.37
45  65.98   106.45
46  68.67   128.76
47  66.88   145.68
48  67.70   116.82
49  69.82   143.62
50  69.09   134.93

Overwriting data/ds_spark_heightweight.txt


In [67]:
from pyspark.sql.types import *
import os

rdd=spark.sparkContext\
    .textFile(os.path.join('data','ds_spark_heightweight.txt'))

In [68]:
myRdd=rdd.map(lambda line:[float(x) for x in line.split('\t')])

In [69]:
myDf=spark.createDataFrame(myRdd,["id","weight","height"])

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 25) (192.168.55.202 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Spark\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 604, in main
  File "C:\Spark\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 596, in process
  File "C:\Spark\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Spark\spark-3.1.2-bin-hadoop3.2\python\pyspark\rdd.py", line 1560, in takeUpToNumLeft
    yield next(iterator)
  File "C:\Spark\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\util.py", line 73, in wrapper
    return f(*args, **kwa

In [70]:

df = spark.createDataFrame([
        (0, "a b c d e spark", 1.0),
        (1, "b d", 0.0),
        (2, "spark f g h", 1.0),
        (3, "hadoop mapreduce", 0.0),
        (4, "my dog has flea problems. help please.",0.0)
    ], ["id", "text", "label"])

In [71]:
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import LogisticRegression

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)

In [72]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [73]:
model = pipeline.fit(df)

In [74]:
myDf = model.transform(df)


In [75]:
myDf.select('label', 'features').show()


+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(262144,[74920,89...|
|  0.0|(262144,[89530,14...|
|  1.0|(262144,[36803,17...|
|  0.0|(262144,[132966,1...|
|  0.0|(262144,[1074,389...|
+-----+--------------------+

